Apparently there's no need to fix sentence size. I'll ignore this notebook for now

In [1]:
import transformers
import datasets

datasets.utils.logging.set_verbosity(datasets.logging.FATAL)

# Checking wikipedia dataset format

In [2]:
tk = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
bos = tk.special_tokens_map["cls_token"]
sep = tk.special_tokens_map["sep_token"]

bos, sep

('[CLS]', '[SEP]')

In [3]:
wiki_ds = datasets.load_dataset("wikipedia", "20220301.en")['train']
wiki_ds = wiki_ds.map(lambda x: {"tokenized_sentence": tk.encode(x['text'])}, num_proc=12)
wiki_ds = wiki_ds.map(lambda x: {"tokenized_length": len(x['tokenized_sentence'])}, num_proc=12)

  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
smaller_than_512 = sum([1 if x <= 512 else 0 for x in wiki_ds['tokenized_length']])
wiki_proportion = smaller_than_512 / len(wiki_ds)

In [5]:
print(f"WIKI - Proportion of sentences with 512 tokens of less: {wiki_proportion}")

WIKI - Proportion of sentences with 512 tokens of less: 0.6714893313948538


In [6]:
book_ds = datasets.load_dataset("bookcorpus")['train']
book_ds = book_ds.map(lambda x: {"tokenized_sentence": tk.encode(x['text'])}, num_proc=12)
book_ds = book_ds.map(lambda x: {"tokenized_length": len(x['tokenized_sentence'])}, num_proc=12)

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
smaller_than_512 = sum([1 if x <= 512 else 0 for x in book_ds['tokenized_length']])
book_proportion = smaller_than_512 / len(book_ds)

In [8]:
print(f"BOOK - Proportion of sentences with 512 tokens of less: {book_proportion}")

BOOK - Proportion of sentences with 512 tokens of less: 0.9999970001713956


# Checking bertimbau dataset format

In [9]:
tk = transformers.AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
bos = tk.special_tokens_map["cls_token"]
sep = tk.special_tokens_map["sep_token"]

bos, sep

('[CLS]', '[SEP]')

In [10]:
brwac_ds = datasets.load_from_disk('../data/processed/brwac-000/')['train']
brwac_ds = brwac_ds.map(lambda x: {"tokenized_sentence": tk.encode(x['processed_text'])}, num_proc=12)
brwac_ds = brwac_ds.map(lambda x: {"tokenized_length": len(x['tokenized_sentence'])}, num_proc=12)

In [11]:
smaller_than_512 = sum([1 if x <= 512 else 0 for x in brwac_ds['tokenized_length']])
brwac_proportion = smaller_than_512 / len(brwac_ds)

In [12]:
print(f"BRWAC - Proportion of sentences with 512 tokens of less: {brwac_proportion}")

BRWAC - Proportion of sentences with 512 tokens of less: 0.5473369177941745


# What happens if we break sentences on their "." signals?

In [13]:
tk.encode("oi.")

[101, 146, 22283, 119, 102]

In [14]:
from collections import Counter

brwac_ds[100]['tokenized_length']
brwac_ds[100]['tokenized_sentence']
c = Counter(brwac_ds[100]['tokenized_sentence'])
c.most_common()

[(117, 104),
 (125, 61),
 (119, 48),
 (179, 33),
 (122, 28),
 (146, 27),
 (253, 24),
 (118, 20),
 (123, 20),
 (1379, 18),
 (222, 17),
 (230, 16),
 (22283, 14),
 (176, 14),
 (2745, 13),
 (173, 12),
 (180, 12),
 (998, 11),
 (22281, 11),
 (259, 10),
 (202, 10),
 (1228, 9),
 (346, 9),
 (171, 9),
 (22326, 9),
 (221, 8),
 (698, 8),
 (4690, 8),
 (9704, 8),
 (533, 7),
 (358, 7),
 (10678, 7),
 (9340, 7),
 (2465, 7),
 (2973, 7),
 (449, 7),
 (11648, 6),
 (503, 6),
 (7252, 6),
 (22288, 6),
 (989, 6),
 (3459, 6),
 (22287, 6),
 (131, 5),
 (229, 5),
 (271, 5),
 (132, 5),
 (320, 5),
 (231, 5),
 (13551, 5),
 (18510, 5),
 (740, 5),
 (298, 4),
 (8785, 4),
 (113, 4),
 (114, 4),
 (453, 4),
 (8224, 4),
 (4178, 4),
 (170, 4),
 (911, 4),
 (4141, 4),
 (4943, 4),
 (22278, 4),
 (7331, 4),
 (1322, 3),
 (4062, 3),
 (596, 3),
 (291, 3),
 (498, 3),
 (325, 3),
 (407, 3),
 (9861, 3),
 (4659, 3),
 (177, 3),
 (1081, 3),
 (2086, 3),
 (744, 3),
 (240, 3),
 (17395, 3),
 (2654, 3),
 (22280, 3),
 (2658, 3),
 (8132, 3),
 (366

In [15]:
brwac_ds[100]['processed_text']

'Os Vigilantes / Gun chung / Eye in the Sky De: Nai-Hoi Yau, Hong-Kong, 2007 Nota: Anotação em 2009: Faz um bom tempo que ouço falar aqui e ali sobre a qualidade dos filmes de ação, policiais, thrillers, feitos em Hong Kong. Este filme Os Vigilantes, que esteve na programação da TV a cabo, é uma prova cabal de que isso que se fala é a mais pura verdade dos fatos. Não sou propriamente um fã de filmes de ação, mas gosto de bons filmes, e também de surpresas, de filmes sobre os quais não sei absolutamente nada – e aí resolvi ver Os Vigilantes para experimentar. Tem talento e competência de sobra. Além de comprovar que o povo da ex-colônia britânica encravada no planeta China domina bem esse tipo de artesanato, Os Vigilantes é outra prova de que aquilo ali de fato é outro mundo, muito distante do nosso. A história se passa na Hong Kong de hoje (o filme é bem recente, de 2007), mas é como se estivéssemos vendo uma ficção científica. É tudo futurista; os diversos departamentos policiais de H

In [16]:
# Pra depois: Quebrar textos muito grandes em sentenças menores usando o "." como âncora.

In [17]:
tk.vocab['.']

119

In [18]:
dataset = brwac_ds['tokenized_sentence']

dataset_list = list()
sentences = list()
sentence = [101]

for tokens in dataset:
    for token in tokens:
        if token == 101 or token == 102:
            continue
        if token == 119:
            sentence.append(102)
            sentences.append(sentence)
            sentence = [101]
        sentence.append(token)
    dataset_list.append(sentences)
    sentences = list()